In [4]:
from pathlib import Path
import torch
import yaml
import pickle

from weight_diffusion.data.data_utils.helper import perform_train_test_validation_split
from weight_diffusion.data.modelzoo_with_latent_dataset import ModelZooWithLatentDataset
from weight_diffusion.ofga.sampling import sample_from_prompt
from weight_diffusion.data.modelzoo_dataset import get_all_directories_for_a_path
from weight_diffusion.execution.util import load_model_from_config

In [3]:
test_models = perform_train_test_validation_split(
    list_to_split=get_all_directories_for_a_path("/Users/alexanderlontke/Documents/Uni/St. Gallen/HS_22_23/integrative_master_project/data/tune_zoo_mnist_uniform"),
    dataset_split_ratios=[7,3],
    split="test"
)

In [5]:
config_path = "/Users/alexanderlontke/Documents/Uni/St. Gallen/HS_22_23/integrative_master_project/weight-diffusion/configs/ldm/hp-ldm.yaml"
with open(config_path, "r") as config_file:
    config = yaml.safe_load(config_file)

dataset = ModelZooWithLatentDataset(
    data_dir=Path(
        "/Users/alexanderlontke/Documents/Uni/St. Gallen/HS_22_23/integrative_master_project/data/tune_zoo_mnist_uniform"
    ),
    split="irrelevant",
    checkpoint_property_of_interest="test_acc",
    openai_coefficient=config["openai_coefficient"],
    number_of_permutations=10,
    device="cpu",
    model_directory_paths=test_models[:10],
    encoder_config=config["encoder_config"],
    tokenizer_config=config["tokenizer_config"],
)

init attn encoder
## encoder -- use index_dict
model: use only positive contrast loss
initialze projection head


Loading Models:   0%|          | 0/10 [00:00<?, ?it/s]

compute 2 random permutations for layer 0 - 0
compute 2 random permutations for layer 1 - 3
compute 2 random permutations for layer 2 - 6
compute 2 random permutations for layer 3 - 9
prepared 10 permutations
prepare permutation dicts


Indexing dataset: 100%|██████████| 10/10 [00:00<00:00, 1873.04it/s]


In [14]:
prompts = [torch.Tensor([[*dataset[10 * (i*5)]["prompt_latent"]]]) for i in range(10)]

In [15]:
model = load_model_from_config(
    config=config,
    ckpt="/Users/alexanderlontke/Desktop/last.ckpt"
)

Loading model from /Users/alexanderlontke/Desktop/last.ckpt
Global Step: 35680
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 387.94 M params.


In [16]:
shape = dataset[123]["checkpoint_latent"].shape
samples = {}
for prompt in prompts:
    samples[str(prompt)] = sample_from_prompt(
        prompt=prompt,
        sampling_steps=50,
        shape=shape,
        guidance_scale=1.0,
        uc=None,
        model=model,
    )

/Users/alexanderlontke/.conda/envs/weight-diffusion/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Data shape for DDIM sampling is torch.Size([1, 700]), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:08<00:00,  2.56s/it]


Data shape for DDIM sampling is torch.Size([1, 700]), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [02:08<00:00,  2.57s/it]


...

Data shape for DDIM sampling is torch.Size([1, 700]), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [01:58<00:00,  2.37s/it]


Data shape for DDIM sampling is torch.Size([1, 700]), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [01:59<00:00,  2.39s/it]


NameError: name 'pickle' is not defined

In [20]:
with open("/Users/alexanderlontke/Desktop/ldm_samples.pkl", "wb") as file:
    pickle.dump(samples, file=file)
# with open("/Users/alexanderlontke/Desktop/ldm_samples.pkl", "rb") as input_file:
#     loaded_samples = pickle.load(input_file)

## Run Evaluation ##

In [ ]:
sampled_weights = encoder.forward_decoder(sampled_weights_latent)
print("################    WEIGHTS       ###############")
print(sampled_weights)
print("################################################")
sampled_checkpoint = generate_checkpoints_from_weights(
    sampled_weights, model_config, layer_list
)
sampled_mnist_model_checkpoints_dict[str(prompt)] = sampled_checkpoint
# Return dictionary containing target metrics for each prompt
targets_dict[str(prompt)] = prompt_statistics